# QMUL Careers Team Python Script
### A detailed guide on how to run this programme.



Each text box describe what the code does in the cell below it.

The first cell imports some of the software that the programme needs, if you click the small play button above, or press Shift and Enter, it will run the cell.

In [3]:
import pandas as pd
import openpyxl
import numpy as np
import os
import ipywidgets as widgets

We need the excel workbook that the programme will use, and within that the excel spreadsheet with the student information. Please name the spreadsheet with the application information: "data" and save it on your computer.

The video below shows how to add this to the programme.

In [4]:
from IPython.display import Video; from ipywidgets import interactive, IntSlider

vid = Video(filename="media/add_applications.mov",data="", width=800, height = 600)
display(vid)

# enter sorting priority


Next, point the programme at the excel sheet just uploaded.

In [7]:
def get_sheet():
    for file in os.listdir('.'):
        if file.endswith(".xlsx"):
            myfile = file
            print(f'{file} has been selected.')            
    try:
        return myfile
    except:
        print('There is no file uploaded')
    

filename = get_sheet()

groups_assigned.xlsx has been selected.


Read the spreadsheet in, and perform some data cleaning.

In [13]:
df = pd.read_excel(filename,"data")
for i in df.columns:
    df.rename(columns={i:i.lower()}, inplace=True)
    print(i)

allocation
id
student number
first name
last name
email
telephone
course
department
faculty
student type
year of study
fee status
nationality
gender
bursary holder
previous round of scp?
scp will run as a mixture of face to face and online learning. please indicate which format you can participate in and we will do our best to match you with an appropriate team and project
scp - which country will you be located in for the duration of the programme wednesday 26th january - wednesday 13th april?
scp - you are required to be available every wednesday afternoon (2pm to 6pm) for the duration of the programme. will you be available on wednesday afternoons?
q1 - please explain what has motivated you to take part in this round of the scp programme
q2 - please detail three skills which you can bring to a scp team. give examples which clearly evidence you have these skills.
q3 - there are 5 of you working together on a team project and you are presenting to the client in 2 weeks. one member of 

Enter the fields that you would like to prioritise. Each field you add will be used to sort the groups.

To stop running the cell type <b>'exit'</b> in the input box.

In [15]:
def add_cols():
    cols_to_sort = []
    count = 2
    while True:
        col = input('Enter the column you would like to add to the sorting algorithm, if there are no more type exit: ')
        if col.lower() == 'exit':
            [print(f'You have exited, your sorting columns are {str(cols_to_sort)}.')]
            break
        elif col in cols_to_sort:
            print('This column has already been added, please add another column or type exit to stop the programme.')
        elif col in df.columns:
            cols_to_sort.append(col)
            print(f'{col} will be used in the sorting algorithm. It will be priority number {count}')
            count+=1
        elif col not in df.columns:
            print(f'{col} is not a valid column. Please enter a valid column, or type exit to stop the programme.')
    return cols_to_sort
    
for_sorting = add_cols()

gender will be used in the sorting algorithm. It will be priority number 2
You have exited, your sorting columns are ['gender'].


Any missing scores will be replaced with the average score of the dataset.

In [ ]:
df['total'].fillna(df['total'].mean(), inplace=True)

Some exploratory data analysis in the next 2 cells

In [ ]:
df['gender'].value_counts()

In [ ]:
df['year of study'].value_counts()

Take only the columns important to establishing the groups. 

In [ ]:
group_df = df[[ 
'student number',
'year of study',
'gender',
'faculty',
'total',
'course',
'personality type']]

'email', 
'Course', 
'YearOfStudy',
'Gender', 
'Bursary holder', 
'Client meetings delivery', 
'Total score',
'Personality type'

In [ ]:
group_df = group_df.assign(groupno='')



#### The Sorting Algorithm

In [ ]:
def assign_group(df):
    df.sort_values(by=for_sorting, ascending=False, inplace=True)
    count = 1
    up = True
    for i,row in group_df.iterrows():

        if count <= round(len(df)/5,0) and up == True:
            df.at[i,'groupno'] = count
            count += 1

        elif count > round(len(df)/5,0) and up == True:
            df.at[i,'groupno'] = count-1
            count-=1
            up = False

        if count > 1 and up == False:
            df.at[i,'groupno'] = count
            count -= 1
            
        elif count == 1 and up == False:
            df.at[i,'groupno'] = count
            up = True
    
    return df

assigned = assign_group(group_df)

    

# Data Analysis

In [14]:
assigned

NameError: name 'assigned' is not defined

In [ ]:
assigned.groupby(['groupno'])[['total']].mean().round(2).min()

In [ ]:
assigned.pivot_table(index=['groupno'], columns=['year of study'], aggfunc='size', fill_value=0)


In [ ]:
assigned.pivot_table(index=['groupno'], columns=['gender'], aggfunc='size', fill_value=0)


### Send the groups back into the original spreadsheet

In [ ]:
subset = assigned[['student number','groupno']]

final = pd.merge(df,subset, how='inner', on='student number')



In [ ]:
book = openpyxl.load_workbook(filename)
writer = pd.ExcelWriter(filename, engine = 'openpyxl')
writer.book = book
final.to_excel(writer, sheet_name='New Sheet Groups', index=False)
writer.save()
writer.close()

## Make sure you download your new spreadsheet, you can copy and paste it into your previous working document.